In [ ]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, col, desc, sum, when, row_number, lit
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("Football Analysis").getOrCreate()

results_url = "https://raw.githubusercontent.com/tnavarrete-iedib/bigdata-24-25/refs/heads/main/results.csv"
goalscorers_url = "https://raw.githubusercontent.com/tnavarrete-iedib/bigdata-24-25/refs/heads/main/goalscorers.csv"

!wget -O results.csv {results_url}
!wget -O goalscorers.csv {goalscorers_url}

results_df = spark.read.csv("results.csv", header=True, inferSchema=True)
goalscorers_df = spark.read.csv("goalscorers.csv", header=True, inferSchema=True)

from pyspark.sql.functions import lower, when, col

goalscorers_df = goalscorers_df.withColumn("own_goal",
                                        when(lower(col("own_goal")) == "true", True)
                                        .otherwise(False))

goalscorers_df = goalscorers_df.withColumn("penalty",
                                        when(lower(col("penalty")) == "true", True)
                                        .otherwise(False))

results_df.createOrReplaceTempView("results")
goalscorers_df.createOrReplaceTempView("goalscorers")

print("Esquema de results_df:")
results_df.printSchema()

print("\nEsquema de goalscorers_df:")
goalscorers_df.printSchema()

print("\nPrimeres files de results_df:")
results_df.show(5)

print("\nPrimeres files de goalscorers_df:")
goalscorers_df.show(5)

# 1. Nombre de gols que ha marcat en Lionel Messi (sense comptar autogols)

# Mètode DataFrame
messi_goals_df = goalscorers_df.filter((col("scorer") == "Lionel Messi") & (col("own_goal") == False)).count()

print("\n1. Nombre de gols que ha marcat en Lionel Messi (sense comptar autogols):")
print(f"DataFrame: {messi_goals_df}")

# Sentència SQL
messi_goals_sql = spark.sql("""
    SELECT COUNT(*) as gols
    FROM goalscorers
    WHERE scorer = 'Lionel Messi' AND own_goal = false
""")

print("SQL: ")
messi_goals_sql.show()

# 2. Llistat dels 5 partits més recents que ha jugat la selecció espanyola

# Mètode DataFrame
spain_matches_df = results_df.filter((col("home_team") == "Spain") | (col("away_team") == "Spain")) \
                           .orderBy(col("date").desc()) \
                           .limit(5)

print("\n2. Llistat dels 5 partits més recents que ha jugat la selecció espanyola:")
print("DataFrame:")
spain_matches_df.show(5)

# Sentència SQL
spain_matches_sql = spark.sql("""
    SELECT *
    FROM results
    WHERE home_team = 'Spain' OR away_team = 'Spain'
    ORDER BY date DESC
    LIMIT 5
""")

print("SQL:")
spain_matches_sql.show(5)

# 3. Nombre de gols que ha marcat Espanya en tota la seva història (des de results)

# Mètode DataFrame
spain_total_goals_df = results_df.filter((col("home_team") == "Spain") | (col("away_team") == "Spain")) \
                               .withColumn("spain_goals",
                                          when(col("home_team") == "Spain", col("home_score"))
                                          .otherwise(col("away_score"))) \
                               .agg(sum("spain_goals").alias("total_goals"))

print("\n3. Nombre de gols que ha marcat Espanya en tota la seva història:")
print("DataFrame:")
spain_total_goals_df.show()

# Sentència SQL
spain_total_goals_sql = spark.sql("""
    SELECT SUM(
        CASE
            WHEN home_team = 'Spain' THEN home_score
            ELSE away_score
        END
    ) as total_goals
    FROM results
    WHERE home_team = 'Spain' OR away_team = 'Spain'
""")

print("SQL:")
spain_total_goals_sql.show()

# 4. Llistat dels 5 màxims golejadors amb la selecció espanyola (sense comptar autogols)

# Mètode DataFrame
top_spanish_scorers_df = goalscorers_df.filter((col("team") == "Spain") & (col("own_goal") == False)) \
                                     .groupBy("scorer") \
                                     .count() \
                                     .orderBy(col("count").desc()) \
                                     .limit(5)

print("\n4. Llistat dels 5 màxims golejadors amb la selecció espanyola:")
print("DataFrame:")
top_spanish_scorers_df.show(5)

# Sentència SQL
top_spanish_scorers_sql = spark.sql("""
    SELECT scorer, COUNT(*) as goals
    FROM goalscorers
    WHERE team = 'Spain' AND own_goal = false
    GROUP BY scorer
    ORDER BY goals DESC
    LIMIT 5
""")

print("SQL:")
top_spanish_scorers_sql.show(5)

# 5. Llistat dels jugadors espanyols que han marcat algun gol de penal en alguna Eurocopa (UEFA Euro), ordenats alfabèticament

# Mètode DataFrame
spanish_euro_penalty_scorers_df = goalscorers_df.join(
    results_df,
    (goalscorers_df["date"] == results_df["date"]) &
    (((goalscorers_df["home_team"] == results_df["home_team"]) & (goalscorers_df["away_team"] == results_df["away_team"])) |
     ((goalscorers_df["home_team"] == results_df["away_team"]) & (goalscorers_df["away_team"] == results_df["home_team"]))),
    "inner"
) \
.filter((col("team") == "Spain") &
        (col("tournament").like("%UEFA Euro%")) &
        (col("penalty") == True)) \
.select("scorer") \
.distinct() \
.orderBy("scorer")

print("\n5. Llistat dels jugadors espanyols que han marcat algun gol de penal en alguna Eurocopa:")
print("DataFrame:")
spanish_euro_penalty_scorers_df.show(100)

# Sentència SQL
spanish_euro_penalty_scorers_sql = spark.sql("""
    SELECT DISTINCT g.scorer
    FROM goalscorers g
    JOIN results r
    ON g.date = r.date
    AND ((g.home_team = r.home_team AND g.away_team = r.away_team)
         OR (g.home_team = r.away_team AND g.away_team = r.home_team))
    WHERE g.team = 'Spain'
    AND r.tournament LIKE '%UEFA Euro%'
    AND g.penalty = true
    ORDER BY g.scorer
""")

print("SQL:")
spanish_euro_penalty_scorers_sql.show(100)

# 6. Llistat dels 5 màxims golejadors de les fases finals dels mundials (FIFA World Cup) (sense comptar autogols)

# Mètode DataFrame
top_world_cup_scorers_df = goalscorers_df.join(
    results_df,
    (goalscorers_df["date"] == results_df["date"]) &
    (((goalscorers_df["home_team"] == results_df["home_team"]) & (goalscorers_df["away_team"] == results_df["away_team"])) |
     ((goalscorers_df["home_team"] == results_df["away_team"]) & (goalscorers_df["away_team"] == results_df["home_team"]))),
    "inner"
) \
.filter((col("tournament").like("%FIFA World Cup%")) &
        (col("own_goal") == False)) \
.groupBy("scorer") \
.count() \
.orderBy(col("count").desc()) \
.limit(5)

print("\n6. Llistat dels 5 màxims golejadors de les fases finals dels mundials:")
print("DataFrame:")
top_world_cup_scorers_df.show(5)

# Sentència SQL
top_world_cup_scorers_sql = spark.sql("""
    SELECT g.scorer, COUNT(*) as goals
    FROM goalscorers g
    JOIN results r
    ON g.date = r.date
    AND ((g.home_team = r.home_team AND g.away_team = r.away_team)
         OR (g.home_team = r.away_team AND g.away_team = r.home_team))
    WHERE r.tournament LIKE '%FIFA World Cup%'
    AND g.own_goal = false
    GROUP BY g.scorer
    ORDER BY goals DESC
    LIMIT 5
""")

print("SQL:")
top_world_cup_scorers_sql.show(5)

--2025-04-20 11:53:28--  https://raw.githubusercontent.com/tnavarrete-iedib/bigdata-24-25/refs/heads/main/results.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3628614 (3.5M) [text/plain]
Saving to: ‘results.csv’

results.csv         100%[===================>]   3.46M  --.-KB/s    in 0.08s   

2025-04-20 11:53:29 (45.9 MB/s) - ‘results.csv’ saved [3628614/3628614]

--2025-04-20 11:53:29--  https://raw.githubusercontent.com/tnavarrete-iedib/bigdata-24-25/refs/heads/main/goalscorers.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
